In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf

rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# set Dropout
do = 0.3

if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# feat = feat[mask]
# API model, if use RNN, first two rnn layer must return_sequences=True


Using TensorFlow backend.


In [2]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(dropout_rate)(net_speech)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_speech) for name in target_names]

    model = Model(input_speech, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(0.1, 0.5, 0.4)
#model = api_model(1, 1, 1)
#model.summary()
# main function for for LOSO
# main function for for LOSO


In [3]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:7869]],
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[7869:]], vad[7869:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_9layer.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 7s 1ms/step - loss: 1.9794 - v_loss: 0.9566 - a_loss: 0.4582 - d_loss: 0.5658 - v_ccc: 0.0434 - a_ccc: 0.5425 - d_ccc: 0.4347 - val_loss: 2.5872 - val_v_loss: 0.8908 - val_a_loss: 0.8649 - val_d_loss: 0.8243 - val_v_ccc: 0.1065 - val_a_ccc: 0.1349 - val_d_ccc: 0.1714
Epoch 2/50
6295/6295 [==============================] - 3s 481us/step - loss: 1.6155 - v_loss: 0.7684 - a_loss: 0.3757 - d_loss: 0.4711 - v_ccc: 0.2311 - a_ccc: 0.6244 - d_ccc: 0.5290 - val_loss: 2.1013 - val_v_loss: 0.8708 - val_a_loss: 0.6094 - val_d_loss: 0.6272 - val_v_ccc: 0.1292 - val_a_ccc: 0.3954 - val_d_ccc: 0.3742
Epoch 3/50
6295/6295 [==============================] - 3s 515us/step - loss: 1.5702 - v_loss: 0.7297 - a_loss: 0.3685 - d_loss: 0.4719 - v_ccc: 0.2701 - a_ccc: 0.6315 - d_ccc: 0.5282 - val_loss: 2.0262 - val_v_loss: 0.8

Epoch 8/50
6295/6295 [==============================] - 3s 548us/step - loss: 1.4859 - v_loss: 0.6525 - a_loss: 0.3654 - d_loss: 0.4676 - v_ccc: 0.3475 - a_ccc: 0.6344 - d_ccc: 0.5323 - val_loss: 2.0713 - val_v_loss: 0.8584 - val_a_loss: 0.5784 - val_d_loss: 0.6337 - val_v_ccc: 0.1384 - val_a_ccc: 0.4251 - val_d_ccc: 0.3653
Epoch 9/50
6295/6295 [==============================] - 3s 547us/step - loss: 1.4666 - v_loss: 0.6402 - a_loss: 0.3621 - d_loss: 0.4637 - v_ccc: 0.3596 - a_ccc: 0.6378 - d_ccc: 0.5360 - val_loss: 2.1114 - val_v_loss: 0.8722 - val_a_loss: 0.5888 - val_d_loss: 0.6504 - val_v_ccc: 0.1239 - val_a_ccc: 0.4133 - val_d_ccc: 0.3514
Epoch 10/50
6295/6295 [==============================] - 3s 536us/step - loss: 1.4561 - v_loss: 0.6345 - a_loss: 0.3602 - d_loss: 0.4605 - v_ccc: 0.3650 - a_ccc: 0.6397 - d_ccc: 0.5392 - val_loss: 2.0611 - val_v_loss: 0.8635 - val_a_loss: 0.5617 - val_d_loss: 0.6339 - val_v_ccc: 0.1323 - val_a_ccc: 0.4394 - val_d_ccc: 0.3672
Epoch 11/50
6295/6295

Epoch 20/50
2170/2170 [==============================] - 0s 147us/step
[0.09948007017374039, 0.5714523792266846, 0.4232480525970459]
2
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 8s 1ms/step - loss: 1.9785 - v_loss: 0.9645 - a_loss: 0.4409 - d_loss: 0.5723 - v_ccc: 0.0350 - a_ccc: 0.5589 - d_ccc: 0.4276 - val_loss: 2.4322 - val_v_loss: 0.8245 - val_a_loss: 0.8149 - val_d_loss: 0.7870 - val_v_ccc: 0.1745 - val_a_ccc: 0.1837 - val_d_ccc: 0.2097
Epoch 2/50
6295/6295 [==============================] - 3s 493us/step - loss: 1.5910 - v_loss: 0.7544 - a_loss: 0.3646 - d_loss: 0.4726 - v_ccc: 0.2458 - a_ccc: 0.6357 - d_ccc: 0.5276 - val_loss: 2.0927 - val_v_loss: 0.8569 - val_a_loss: 0.6274 - val_d_loss: 0.6124 - val_v_ccc: 0.1432 - val_a_ccc: 0.3757 - val_d_ccc: 0.3884
Epoch 3/50
6295/6295 [==============================] - 3s 517us/step - loss: 1.5547 - v_loss: 0.7154 - a_loss: 0.3634 - d_loss: 0.4755 - v_ccc: 0.2845 - a_ccc: 0.6365

6295/6295 [==============================] - 3s 553us/step - loss: 1.4260 - v_loss: 0.6108 - a_loss: 0.3524 - d_loss: 0.4629 - v_ccc: 0.3891 - a_ccc: 0.6476 - d_ccc: 0.5372 - val_loss: 2.0326 - val_v_loss: 0.8573 - val_a_loss: 0.5492 - val_d_loss: 0.6262 - val_v_ccc: 0.1423 - val_a_ccc: 0.4519 - val_d_ccc: 0.3732
Epoch 12/50
6295/6295 [==============================] - 3s 523us/step - loss: 1.4189 - v_loss: 0.6144 - a_loss: 0.3491 - d_loss: 0.4560 - v_ccc: 0.3859 - a_ccc: 0.6513 - d_ccc: 0.5439 - val_loss: 2.0627 - val_v_loss: 0.8630 - val_a_loss: 0.5698 - val_d_loss: 0.6276 - val_v_ccc: 0.1351 - val_a_ccc: 0.4306 - val_d_ccc: 0.3716
Epoch 13/50
2170/2170 [==============================] - 0s 155us/step
[0.09717285633087158, 0.5696130394935608, 0.45120322704315186]
4
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 9s 2ms/step - loss: 2.0161 - v_loss: 0.9820 - a_loss: 0.4516 - d_loss: 0.5819 - v_ccc: 0.0181 - a_ccc: 0.5481 - d_ccc:

6295/6295 [==============================] - 3s 512us/step - loss: 1.4653 - v_loss: 0.6313 - a_loss: 0.3679 - d_loss: 0.4658 - v_ccc: 0.3685 - a_ccc: 0.6320 - d_ccc: 0.5341 - val_loss: 2.0713 - val_v_loss: 0.8532 - val_a_loss: 0.5673 - val_d_loss: 0.6494 - val_v_ccc: 0.1456 - val_a_ccc: 0.4309 - val_d_ccc: 0.3522
Epoch 10/50
6295/6295 [==============================] - 3s 523us/step - loss: 1.4451 - v_loss: 0.6246 - a_loss: 0.3610 - d_loss: 0.4589 - v_ccc: 0.3751 - a_ccc: 0.6388 - d_ccc: 0.5409 - val_loss: 2.0061 - val_v_loss: 0.8370 - val_a_loss: 0.5443 - val_d_loss: 0.6194 - val_v_ccc: 0.1586 - val_a_ccc: 0.4547 - val_d_ccc: 0.3805
Epoch 11/50
6295/6295 [==============================] - 4s 565us/step - loss: 1.4256 - v_loss: 0.6065 - a_loss: 0.3630 - d_loss: 0.4563 - v_ccc: 0.3942 - a_ccc: 0.6368 - d_ccc: 0.5434 - val_loss: 1.9911 - val_v_loss: 0.8362 - val_a_loss: 0.5437 - val_d_loss: 0.6046 - val_v_ccc: 0.1609 - val_a_ccc: 0.4554 - val_d_ccc: 0.3926
Epoch 12/50
6295/6295 [========

...

Epoch 13/50
2170/2170 [==============================] - 0s 200us/step
[0.09968522191047668, 0.5550084114074707, 0.4318745732307434]
12
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 12s 2ms/step - loss: 1.9404 - v_loss: 0.9430 - a_loss: 0.4417 - d_loss: 0.5543 - v_ccc: 0.0564 - a_ccc: 0.5579 - d_ccc: 0.4453 - val_loss: 2.6394 - val_v_loss: 0.9007 - val_a_loss: 0.8722 - val_d_loss: 0.8595 - val_v_ccc: 0.0971 - val_a_ccc: 0.1264 - val_d_ccc: 0.1372
Epoch 2/50
6295/6295 [==============================] - 3s 538us/step - loss: 1.6060 - v_loss: 0.7636 - a_loss: 0.3701 - d_loss: 0.4721 - v_ccc: 0.2362 - a_ccc: 0.6298 - d_ccc: 0.5280 - val_loss: 2.1228 - val_v_loss: 0.8524 - val_a_loss: 0.6292 - val_d_loss: 0.6448 - val_v_ccc: 0.1476 - val_a_ccc: 0.3744 - val_d_ccc: 0.3552
Epoch 3/50
6295/6295 [==============================] - 3s 525us/step - loss: 1.5652 - v_loss: 0.7250 - a_loss: 0.3717 - d_loss: 0.4691 - v_ccc: 0.2750 - a_ccc: 0.62

6295/6295 [==============================] - 3s 541us/step - loss: 1.4714 - v_loss: 0.6427 - a_loss: 0.3620 - d_loss: 0.4672 - v_ccc: 0.3573 - a_ccc: 0.6382 - d_ccc: 0.5331 - val_loss: 1.9976 - val_v_loss: 0.8459 - val_a_loss: 0.5422 - val_d_loss: 0.6058 - val_v_ccc: 0.1500 - val_a_ccc: 0.4587 - val_d_ccc: 0.3938
Epoch 10/50
6295/6295 [==============================] - 3s 550us/step - loss: 1.4543 - v_loss: 0.6337 - a_loss: 0.3583 - d_loss: 0.4629 - v_ccc: 0.3664 - a_ccc: 0.6419 - d_ccc: 0.5373 - val_loss: 2.0152 - val_v_loss: 0.8680 - val_a_loss: 0.5432 - val_d_loss: 0.5984 - val_v_ccc: 0.1292 - val_a_ccc: 0.4582 - val_d_ccc: 0.3974
Epoch 11/50
6295/6295 [==============================] - 3s 525us/step - loss: 1.4462 - v_loss: 0.6256 - a_loss: 0.3581 - d_loss: 0.4618 - v_ccc: 0.3740 - a_ccc: 0.6417 - d_ccc: 0.5382 - val_loss: 2.0621 - val_v_loss: 0.8775 - val_a_loss: 0.5596 - val_d_loss: 0.6234 - val_v_ccc: 0.1223 - val_a_ccc: 0.4419 - val_d_ccc: 0.3737
Epoch 12/50
6295/6295 [========

Epoch 21/50
6295/6295 [==============================] - 3s 544us/step - loss: 1.3240 - v_loss: 0.5277 - a_loss: 0.3460 - d_loss: 0.4505 - v_ccc: 0.4724 - a_ccc: 0.6541 - d_ccc: 0.5495 - val_loss: 2.0347 - val_v_loss: 0.8440 - val_a_loss: 0.5564 - val_d_loss: 0.6379 - val_v_ccc: 0.1565 - val_a_ccc: 0.4450 - val_d_ccc: 0.3638
Epoch 22/50
6295/6295 [==============================] - 4s 578us/step - loss: 1.3208 - v_loss: 0.5376 - a_loss: 0.3384 - d_loss: 0.4450 - v_ccc: 0.4629 - a_ccc: 0.6614 - d_ccc: 0.5550 - val_loss: 1.9936 - val_v_loss: 0.8191 - val_a_loss: 0.5461 - val_d_loss: 0.6273 - val_v_ccc: 0.1804 - val_a_ccc: 0.4546 - val_d_ccc: 0.3714
Epoch 23/50
6295/6295 [==============================] - 4s 595us/step - loss: 1.3255 - v_loss: 0.5310 - a_loss: 0.3498 - d_loss: 0.4447 - v_ccc: 0.4687 - a_ccc: 0.6504 - d_ccc: 0.5554 - val_loss: 2.0208 - val_v_loss: 0.8433 - val_a_loss: 0.5400 - val_d_loss: 0.6372 - val_v_ccc: 0.1567 - val_a_ccc: 0.4590 - val_d_ccc: 0.3635
Epoch 24/50
2170/21

6295/6295 [==============================] - 3s 551us/step - loss: 1.4692 - v_loss: 0.6425 - a_loss: 0.3618 - d_loss: 0.4645 - v_ccc: 0.3573 - a_ccc: 0.6380 - d_ccc: 0.5355 - val_loss: 1.9866 - val_v_loss: 0.8465 - val_a_loss: 0.5418 - val_d_loss: 0.5986 - val_v_ccc: 0.1505 - val_a_ccc: 0.4618 - val_d_ccc: 0.4010
Epoch 10/50
6295/6295 [==============================] - 3s 548us/step - loss: 1.4660 - v_loss: 0.6424 - a_loss: 0.3617 - d_loss: 0.4614 - v_ccc: 0.3574 - a_ccc: 0.6381 - d_ccc: 0.5384 - val_loss: 2.0016 - val_v_loss: 0.8492 - val_a_loss: 0.5463 - val_d_loss: 0.6114 - val_v_ccc: 0.1486 - val_a_ccc: 0.4607 - val_d_ccc: 0.3892
Epoch 11/50
6295/6295 [==============================] - 4s 574us/step - loss: 1.4438 - v_loss: 0.6285 - a_loss: 0.3529 - d_loss: 0.4624 - v_ccc: 0.3715 - a_ccc: 0.6471 - d_ccc: 0.5375 - val_loss: 1.9950 - val_v_loss: 0.8404 - val_a_loss: 0.5372 - val_d_loss: 0.6180 - val_v_ccc: 0.1580 - val_a_ccc: 0.4654 - val_d_ccc: 0.3816
Epoch 12/50
6295/6295 [========

6295/6295 [==============================] - 4s 685us/step - loss: 1.4899 - v_loss: 0.6594 - a_loss: 0.3649 - d_loss: 0.4663 - v_ccc: 0.3410 - a_ccc: 0.6354 - d_ccc: 0.5338 - val_loss: 1.9865 - val_v_loss: 0.8517 - val_a_loss: 0.5301 - val_d_loss: 0.6061 - val_v_ccc: 0.1467 - val_a_ccc: 0.4731 - val_d_ccc: 0.3937
Epoch 8/50
6295/6295 [==============================] - 4s 661us/step - loss: 1.4842 - v_loss: 0.6506 - a_loss: 0.3644 - d_loss: 0.4695 - v_ccc: 0.3493 - a_ccc: 0.6356 - d_ccc: 0.5309 - val_loss: 2.0440 - val_v_loss: 0.8575 - val_a_loss: 0.5626 - val_d_loss: 0.6243 - val_v_ccc: 0.1410 - val_a_ccc: 0.4392 - val_d_ccc: 0.3759
Epoch 9/50
6295/6295 [==============================] - 4s 610us/step - loss: 1.4686 - v_loss: 0.6470 - a_loss: 0.3601 - d_loss: 0.4614 - v_ccc: 0.3531 - a_ccc: 0.6398 - d_ccc: 0.5385 - val_loss: 2.0613 - val_v_loss: 0.8607 - val_a_loss: 0.5625 - val_d_loss: 0.6390 - val_v_ccc: 0.1384 - val_a_ccc: 0.4376 - val_d_ccc: 0.3627
Epoch 10/50
6295/6295 [==========